In [ ]:
# IMPORTS
import pandas as pd
import urllib
import json
import shapefile
from shapely.geometry import shape, Point
from shapely.ops import transform
import pyproj

In [ ]:
# READ THE EMISSIONS DATASET

df_emissions=pd.read_csv("./uk-local-authority-ghg-emissions-2020-dataset.csv")
df_la = df_emissions.drop_duplicates(subset=["Local Authority"], keep="last")[["Local Authority Code", "Region", "Local Authority", "Area (km2)", "Mid-year Population (thousands)", "Calendar Year"]];
df_la.drop(df_la[df_la["Region"]=="London"].index, inplace=True);
df_la = df_la.sort_values(["Local Authority Code", "Calendar Year", "Area (km2)"])
df_la

In [ ]:
# READ ROAD LENGTHS DATASET

df_roads = pd.read_csv("./rdl0202.csv", skiprows=6)
df_roads = df_roads[["ONS Area \nCode", "Region", "Local Authority", "All 'A' roads", "Total road length "]]
df_roads.drop(df_roads[df_roads["Region"]=="London"].index, inplace=True);
df_roads=df_roads.sort_values(["ONS Area \nCode"])
df_roads

In [ ]:
# FETCH COUNT SITES FROM HIGHWAYS ENGLAND API
url_text = "https://webtris.highwaysengland.co.uk/api/v1/sites"
with urllib.request.urlopen(url_text) as url:
    data = json.loads(url.read().decode())
sites = data['sites']

# FILTER TO ONLY CONSIDER A-ROADS
def isARoad(site):
    return site["Description"].startswith("A")
aroads_iterator = filter(isARoad, sites)
aroad_sites = list(aroads_iterator)

In [ ]:
# READ THE SHAPEFILE
#shapefile_location = './GLTLA_DEC_2022_EW_BFC_7755317155209021260/GLTLA_DEC_2022_EW_BFC.shp'
shapefile_location = "./UK_LocalDistrictAuthorities2023/LAD_MAY_2023_UK_BFC.shp"
shp = shapefile.Reader(shapefile_location)
all_shapes = shp.shapes() # get all the polygons
all_records = shp.records()

# DEFINE PROJECTION TO CONVERT SHAPEFILE COORDINATES TO LONG/LAT VALUES
project = pyproj.Transformer.from_proj(
    pyproj.Proj('epsg:27700'), # source coordinate system
    pyproj.Proj('epsg:4326')) # destination coordinate system

def assignSiteToLA(site, dictionary):
    for i in range(0,len(all_shapes)):
        boundary = transform(project.transform, shape(all_shapes[i]))
        if boundary.contains(Point(site["Latitude"], site["Longitude"])):
            la_code =  all_records[i][0]
            dictionary[la_code].append(site)
            break

# DATA STRUCTURE TO STORE COUNT SITES AND THEIR LA'S
la_dictionary = {record[0]: [] for record in all_records}

# ASSIGN COUNT SITES TO LOCAL AUTHORITY (TAKES ~2 HOURS TO RUN)
for index, site in enumerate(aroad_sites):
    assignSiteToLA(site, la_dictionary)
    print("DONE ", index + 1, " sites of ", len(aroad_sites), ".\t", int(float(100)*index/len(aroad_sites))), "%%."

# SAVE DATA STRUCTURE TO JSON FILE TO AVOID RERUNNING
with open('la_countsite_dict.json', 'w') as dictionary_json:
    json.dump(la_dictionary, dictionary_json)